### Find Word Images And Copy

In [1]:
import os
import sys
import pandas as pd
import numpy as np
import re
import glob
from pathlib import Path
import shutil
from os.path import isfile, join

In [2]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

file_ext = 1000

# image result parameter
select_image_num = 1  # 10 image sample number big and equal this number
take_image_sample_num = 1000  # 50

In [3]:
word_lemma_data_path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Lemma Stem POS/Result/3-2-Word In Visual Genome Merge"

word_lemma_data_path2 = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Lemma Stem POS/Result/3-3-Find Word Images Mask Fade"

path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Lemma Stem POS/Result/3-4-Find Word Images And Copy"

#Path(path).mkdir(parents=True, exist_ok=True)

In [4]:
def word_group_dataframe(df, search_list, target_column, sample_num):
    '''
    word_group_dataframe(df_youtube_sentence, search_list, "sentence", 6)\n
    df_youtube_sentence is dataframe and "sentence" is its column for external searching_list
    ''' 
    df_search_result = pd.DataFrame()
    for j in search_list:
        df_select = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=False, regex=True)]
        #df_select.sort_values(f"{target_column}",key=lambda x:x.str.len(), inplace=True).head(sample_num)
        df_select = df_select.sort_values(f"{target_column}",key=lambda x:x.str.len()).head(sample_num)               
        df_select.insert(0,"search_string",j)
        df_search_result = pd.concat([df_search_result,df_select], axis=0)
    df_search_result.reset_index(inplace=True, drop=True)
    
    return df_search_result

In [5]:
def word_group_dataframe_all(df, search_list, target_column):
    '''
    word_group_dataframe(df_youtube_sentence, search_list, "sentence", 6)\n
    df_youtube_sentence is dataframe and "sentence" is its column for external searching_list
    ''' 
    df_search_result = pd.DataFrame()
    for j in search_list:
        df_select = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=False, regex=True)]
        #df_select.sort_values(f"{target_column}",key=lambda x:x.str.len(), inplace=True)
        df_select = df_select.sort_values(f"{target_column}",key=lambda x:x.str.len())
        df_select.insert(0,"search_string",j)
        df_search_result = pd.concat([df_search_result,df_select], axis=0)        
    df_search_result.reset_index(inplace=True, drop=True)
    
    return df_search_result

In [6]:
def take_dataframe_word_sample_from_sorting(df_source, word_list, word_source_column, sort_target_column, sort_ascending=True, sample_num=50):
    '''take_dataframe_word_sample_from_sorting(df_source, word_list, word_source_column, sort_target_column, sort_ascending=True, sample_num=50)\n
    df_source is a dataframe and word_list is equal in word_source_column. Then sort_target_column is sorting according to sort_ascending condition.\n
    Finally, taking sample_num each word_list values.\n 
    ex.\n
    take_dataframe_word_sample_from_sorting(df_genome_word_lemma_concat, word_list, "word", "search_text", sort_ascending=True, sample_num=50)
    '''
    df_search_result = pd.DataFrame()
    for word in word_list:
        df_select = df_source[df_source[f"{word_source_column}"] == word]
        df_select = df_select.sort_values(f"{sort_target_column}",key=lambda x:x.str.len(), ascending=sort_ascending).head(sample_num)
        df_search_result = pd.concat([df_search_result,df_select], axis=0)
    
    df_search_result.reset_index(inplace=True, drop=True)
    
    return df_search_result

In [7]:
#def create_word_folder_and_copy_image(df_source, word_list, word_source_column, image_id_column, image_folder_path, output_path_folder):
#    '''create_word_folder_and_copy_image(df_source, word_list, word_source_column, image_id_column, image_folder_path, output_path_folder)\n
#    df_source is a dataframe and word_list is equal in word_source_column. Then word image id search in image_id_column and image copy to\n
#    output_path_folder from image_folder_path.\n
#    ex.\n
#    image_path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Lemma Stem POS/Data/Visual Genome/images"\n
#    output_path_folder = "/home/kurubal/Downloads/temp folder"\n
#    create_word_folder_and_copy_image(df_sample_result, word_list, "word", "image_id", image_path, output_path_folder)
#    '''
#    df_search_result = pd.DataFrame()
#    for word in word_list:
#        path = f"{output_path_folder}/{word}"
#        Path(path).mkdir(parents=True, exist_ok=True)        
#        df_select = df_source[df_source[f"{word_source_column}"] == word]
#        for image_id in df_select[f"{image_id_column}"]:
#            image_file = glob.glob(f"{image_folder_path}/*/{image_id}.jpg")
#            for l in image_file:
#                source = l # source directory
#                destination = path
#                shutil.copy2(source, destination)

In [27]:
def create_word_folder_and_copy_image(df_source, word_list, word_source_column, image_id_column, opt_column, image_folder_path, mask_folder_path, output_path_folder):
    '''create_word_folder_and_copy_image(df_source, word_list, word_source_column, image_id_column, image_folder_path, mask_folder_path, output_path_folder)\n
    df_source is a dataframe and word_list is equal in word_source_column. Then word image id search in image_id_column and image copy to\n
    output_path_folder from mask_folder_path (search in firstly) and image_folder_path. opt_column used for additional features\n
    ex.\n
    image_path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Lemma Stem POS/Data/Visual Genome/images"\n
    output_path_folder = "/home/kurubal/Downloads/temp folder"\n
    create_word_folder_and_copy_image(df_sample_result, word_list, "word", "image_id", "num", image_path, mask_image_path output_path_folder)
    '''
    df_search_result = pd.DataFrame()
    for word in word_list:
        try:
            path = f"{output_path_folder}/{word}"
            Path(path).mkdir(parents=True, exist_ok=True)        
            df_var = df_source[df_source[f"{word_source_column}"] == word]
            df_select = df_var.drop_duplicates(subset=[f"{image_id_column}"])  # ????
            imageid_list = list(df_select[f"{image_id_column}"])
            opt_column_list = list(df_select[f"{opt_column}"])
            data_list_zip = zip(imageid_list, opt_column_list)
            for image_id, opt in data_list_zip:
                opt_num = int(opt)

                #if isfile(f"{image_folder_path}/VG_Mask_Blur/{image_id}_{opt_num}_mask_blur.jpg"):  # ?????                
                #    image_file = glob.glob(f"{image_folder_path}/*/{image_id}_{opt_num}_mask_blur.jpg")

                if isfile(f"{mask_folder_path}/{image_id}_{opt_num}.jpg"):  # ?????                
                    image_file = glob.glob(f"{mask_folder_path}/{image_id}_{opt_num}.jpg")
                    for l in image_file:
                        source = l # source directory
                        destination = path
                        shutil.copy2(source, destination)
                else:
                    image_file = glob.glob(f"{image_folder_path}/*/{image_id}.jpg")
                    for l in image_file:
                        source = l # source directory
                        destination = path
                        shutil.copy2(source, destination)
        except:
            pass

In [9]:
image_path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Lemma Stem POS/Data/Visual Genome/images"
mask_image_path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Lemma Stem POS/Data/Visual Genome/images/VG_Mask_Blur"

In [10]:
image_folder_list = glob.glob(f"{image_path}/*")
image_folder_list

['/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Turkish/Lemma Stem POS/Data/Visual Genome/images/VG_100K',
 '/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Turkish/Lemma Stem POS/Data/Visual Genome/images/VG_100K_2',
 '/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Turkish/Lemma Stem POS/Data/Visual Genome/images/VG_Mask_Blur']

#### Visual Genome Word Lemma Translate Data

In [11]:
Pos_Tag = "NOUN"

In [24]:
output_path_folder = f"/home/kurubal/Downloads/Image_Select/{Pos_Tag}"

Path(output_path_folder).mkdir(parents=True, exist_ok=True)

In [13]:
df_genome_word_lemma_concat_coor = pd.read_csv(f"{word_lemma_data_path2}/Visual_Genome_{file_ext}_Word_Lemma_Coordinate_Search_Result.csv")
df_genome_word_lemma_concat_coor = df_genome_word_lemma_concat_coor.drop(["height","width","x_koor","y_koor"], axis=1)
df_genome_word_lemma_concat_coor

,num_x,word,lemma.spacy,stem,word_en_translate,lemma_en_translate,image_id,num_y
0,2920126305,bir,bir,bir,a,a,2390994,2920126305
1,129190150113,bir,bir,bir,a,a,2348965,129190150113
2,123317182233,bir,bir,bir,a,a,2348965,123317182233
3,4615142168,bir,bir,bir,a,a,2349861,4615142168
4,54188097,bir,bir,bir,a,a,2349866,54188097
...,...,...,...,...,...,...,...,...
95335,2034269302,çekilin,çek,çek,withdraw,check,2318219,2034269302
95336,4612198435,çekilin,çek,çek,withdraw,check,2333148,4612198435
95337,276026737,çekilin,çek,çek,withdraw,check,2317616,276026737
95338,376517459,çekilin,çek,çek,withdraw,check,2400604,376517459


In [14]:
df_genome_word_lemma_concat = pd.read_csv(f"{word_lemma_data_path}/Visual_Genome_{file_ext}_Word_Lemma_Search_Result.csv")
df_genome_word_lemma_concat

,POS1,POS2,word,lemma.spacy,stem,word_en_translate,lemma_en_translate,frequency,search_text,image_id,num
0,NUM,NaN,bir,bir,bir,a,a,18835735,a,2390994,2920126305
1,NUM,NaN,bir,bir,bir,a,a,18835735,a,2348965,129190150113
2,NUM,NaN,bir,bir,bir,a,a,18835735,a,2348965,123317182233
3,NUM,NaN,bir,bir,bir,a,a,18835735,a,2349861,4615142168
4,NUM,NaN,bir,bir,bir,a,a,18835735,a,2349866,54188097
...,...,...,...,...,...,...,...,...,...,...,...
192717,VERB,NaN,çekilin,çek,çek,withdraw,check,69201,airport check in kiosks,2317616,276026737
192718,VERB,NaN,çekilin,çek,çek,withdraw,check,69201,red check of tablecloth,2400604,376517459
192719,VERB,NaN,çekilin,çek,çek,withdraw,check,69201,a check is on the table,2386272,95132301236
192720,VERB,NaN,çekilin,çek,çek,withdraw,check,69201,cleats with white check,2371210,2211405194


In [15]:
df_genome_word_lemma_concat = df_genome_word_lemma_concat[df_genome_word_lemma_concat["POS1"] == Pos_Tag]
df_genome_word_lemma_concat

,POS1,POS2,word,lemma.spacy,stem,word_en_translate,lemma_en_translate,frequency,search_text,image_id,num
844,NOUN,NaN,şu,şu,şu,this,this,1377715,this,525,3323001266
845,NOUN,NaN,şu,şu,şu,this,this,1377715,this,2350323,127159144184
846,NOUN,NaN,şu,şu,şu,this,this,1377715,this,2350323,622869100
847,NOUN,NaN,şu,şu,şu,this,this,1377715,this,2350323,547564258
848,NOUN,NaN,şu,şu,şu,this,this,1377715,this,2350323,77120262298
...,...,...,...,...,...,...,...,...,...,...,...
192518,NOUN,NaN,sırada,sıra,sıra,in line,desk,69586,desk lamp,2354353,64862790
192519,NOUN,NaN,sırada,sıra,sıra,in line,desk,69586,white desk,2389995,2545371147
192520,NOUN,NaN,sırada,sıra,sıra,in line,desk,69586,glass desk,2389540,4343170199
192521,NOUN,NaN,sırada,sıra,sıra,in line,desk,69586,small desk,2348802,180101241147


In [16]:
df_genome_word_lemma_concat_select = df_genome_word_lemma_concat.groupby("word")["image_id"].apply(lambda x: x.count()>=select_image_num).reset_index()
df_genome_word_lemma_concat_select

,word,image_id
0,acele,True
1,adam,True
2,adama,True
3,adamlar,True
4,adamı,True
...,...,...
411,şeyin,True
412,şeyler,True
413,şeyleri,True
414,şimdiye,True


In [17]:
word_select_list = df_genome_word_lemma_concat_select[df_genome_word_lemma_concat_select["image_id"] == True]["word"].to_list()
#word_select_list

In [18]:
df_genome_word_lemma_concat_select_result = df_genome_word_lemma_concat[df_genome_word_lemma_concat["word"].isin(word_select_list)]
df_genome_word_lemma_concat_select_result

,POS1,POS2,word,lemma.spacy,stem,word_en_translate,lemma_en_translate,frequency,search_text,image_id,num
844,NOUN,NaN,şu,şu,şu,this,this,1377715,this,525,3323001266
845,NOUN,NaN,şu,şu,şu,this,this,1377715,this,2350323,127159144184
846,NOUN,NaN,şu,şu,şu,this,this,1377715,this,2350323,622869100
847,NOUN,NaN,şu,şu,şu,this,this,1377715,this,2350323,547564258
848,NOUN,NaN,şu,şu,şu,this,this,1377715,this,2350323,77120262298
...,...,...,...,...,...,...,...,...,...,...,...
192518,NOUN,NaN,sırada,sıra,sıra,in line,desk,69586,desk lamp,2354353,64862790
192519,NOUN,NaN,sırada,sıra,sıra,in line,desk,69586,white desk,2389995,2545371147
192520,NOUN,NaN,sırada,sıra,sıra,in line,desk,69586,glass desk,2389540,4343170199
192521,NOUN,NaN,sırada,sıra,sıra,in line,desk,69586,small desk,2348802,180101241147


In [19]:
df_genome_word_lemma_concat_select_result_coor_merge = pd.merge(df_genome_word_lemma_concat,df_genome_word_lemma_concat_coor,how="left", on=["word","image_id","word_en_translate","lemma_en_translate","lemma.spacy","stem"])
#df_genome_word_lemma_concat_select_result_coor_merge = pd.merge(df_genome_word_lemma_concat, df_genome_word_lemma_concat_coor,how="left", on=["word","image_id"])
df_genome_word_lemma_concat_select_result_coor_merge = df_genome_word_lemma_concat_select_result_coor_merge.drop_duplicates()
#df_genome_word_lemma_concat_select_result_coor_merge["num"] = df_genome_word_lemma_concat_select_result_coor_merge["num"].fillna(0)
df_genome_word_lemma_concat_select_result_coor_merge

,POS1,POS2,word,lemma.spacy,stem,word_en_translate,lemma_en_translate_x,frequency,search_text,image_id,num,num_x,lemma_en_translate_y,num_y
0,NOUN,NaN,şu,şu,şu,this,this,1377715,this,525,3323001266,3323001266,this,3323001266
1,NOUN,NaN,şu,şu,şu,this,this,1377715,this,2350323,127159144184,127159144184,this,127159144184
2,NOUN,NaN,şu,şu,şu,this,this,1377715,this,2350323,127159144184,622869100,this,622869100
3,NOUN,NaN,şu,şu,şu,this,this,1377715,this,2350323,127159144184,547564258,this,547564258
4,NOUN,NaN,şu,şu,şu,this,this,1377715,this,2350323,127159144184,77120262298,this,77120262298
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122593,NOUN,NaN,sırada,sıra,sıra,in line,desk,69586,desk lamp,2354353,64862790,64862790,desk,64862790
122594,NOUN,NaN,sırada,sıra,sıra,in line,desk,69586,white desk,2389995,2545371147,2545371147,desk,2545371147
122595,NOUN,NaN,sırada,sıra,sıra,in line,desk,69586,glass desk,2389540,4343170199,4343170199,desk,4343170199
122596,NOUN,NaN,sırada,sıra,sıra,in line,desk,69586,small desk,2348802,180101241147,180101241147,desk,180101241147


In [20]:
df_genome_word_lemma_concat_select_result_coor_merge["word"].nunique()

416

In [21]:
word_list = list(set(df_genome_word_lemma_concat_select_result_coor_merge["word"]))
lemma_list = list(set(df_genome_word_lemma_concat_select_result_coor_merge["lemma.spacy"]))

In [22]:
df_genome_sample_result = take_dataframe_word_sample_from_sorting(df_genome_word_lemma_concat_select_result_coor_merge, word_list, "word", 
                                                                    "search_text", sort_ascending=True, sample_num=take_image_sample_num)
df_genome_sample_result

,POS1,POS2,word,lemma.spacy,stem,word_en_translate,lemma_en_translate_x,frequency,search_text,image_id,num,num_x,lemma_en_translate_y,num_y
0,NOUN,NaN,gayet,gayet,gayet,very,very,91951,very,2409001,6449900,6449900,very,6449900
1,NOUN,NaN,gayet,gayet,gayet,very,very,91951,very,2322453,1426196352,1426196352,very,1426196352
2,NOUN,NaN,gayet,gayet,gayet,very,very,91951,very,2316484,237437520,237437520,very,237437520
3,NOUN,NaN,gayet,gayet,gayet,very,very,91951,very,2410754,12374640,12374640,very,12374640
4,NOUN,NaN,gayet,gayet,gayet,very,very,91951,very,2404606,342987440,272887440,very,272887440
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122532,NOUN,NaN,çoktan,çok,çok,already,lots,82798,where are the pizzas already made at in the pi...,2395361,0,NaN,NaN,NaN
122533,NOUN,NaN,çoktan,çok,çok,already,lots,82798,how do we know the woman has already begun the...,2366264,0,NaN,NaN,NaN
122534,NOUN,NaN,çoktan,çok,çok,already,lots,82798,what type of shape is the slice of pizza alrea...,2372381,0,NaN,NaN,NaN
122535,NOUN,NaN,çoktan,çok,çok,already,lots,82798,what corner of the photo shows field that has ...,2413858,0,NaN,NaN,NaN


In [23]:
df_genome_sample_result["word"].nunique()

416

In [ ]:
#for word in word_list:
#    path = f"{output_path_folder}/ALL_{Pos_Tag}_FOLDER/{word}"
#    Path(path).mkdir(parents=True, exist_ok=True)

In [58]:
# for word
#create_word_folder_and_copy_image(df_genome_sample_result, word_list, "word", "image_id", "num", image_path, mask_image_path ,output_path_folder)

In [ ]:
# create folder
for lemma in lemma_list:
    path = f"{output_path_folder}/ALL_{Pos_Tag}_FOLDER/{lemma}"
    Path(path).mkdir(parents=True, exist_ok=True)
    df_var = df_genome_sample_result[df_genome_sample_result["lemma.spacy"] == f"{lemma}"]
    for word in set(df_var["word"]):
        path2 = f"{path}/{word}"
        Path(path2).mkdir(parents=True, exist_ok=True)

In [28]:
# for lemma
create_word_folder_and_copy_image(df_genome_sample_result, lemma_list, "lemma.spacy", "image_id", "num", image_path, mask_image_path ,output_path_folder)